## MLOps Agent

## The scope of this Notebook is to provide instructions on how to use DataRobot's MLOps Agents.


resources:

* Agents webinar: https://community.datarobot.com/t5/learning-sessions/monitoring-all-your-models-with-datarobot-agents/ba-p/7732
* Working with Remote Models: https://community.datarobot.com/t5/resources/working-with-remote-models/ta-p/7517
* Deploy in SageMaker and Monitor with MLOps Agents: https://community.datarobot.com/t5/resources/deploy-in-sagemaker-and-monitor-with-mlops-agents/ta-p/5771


In [ ]:
%%sh
git clone https://github.com/timsetsfire/agents-plus-challengers.git

Cloning into 'agents-plus-challengers'...


In [ ]:
%%sh
pip install -r /content/agents-plus-challengers/remote-model/requirements.txt -q
pip install datarobot urllib3==1.26.4 -U -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import pandas as pd
import bson
import time
import sys
import pprint
import os
import datarobot as dr
import yaml
import subprocess 
import urllib3
import requests
import json
import bson
sys.path.append("/content/agents-plus-challengers/remote-model")

In [ ]:
df = pd.read_csv(
    "/content/agents-plus-challengers/data/DR_Demo_SCMS.csv"
    ).sample(frac=0.3)
target = "Late_delivery"
df[target].describe()

df["AssociationID"] = 1
df["AssociationID"] = df["AssociationID"].apply(lambda x: str(bson.ObjectId()))

sampled_df = df.sample(frac = 0.20)

## Grab agents tarball from DataRobot

In [ ]:
token = "NjE2NDQwMGUwZGFlOTllZTA4ZWEyMzUzOkg3M25tck1pWWM2eWNvSkUzbGQzV1dYSEljNjhXSmV6dHZXQkFHQUc4b0k9"
endpoint = "https://app.datarobot.com"
## connect to DataRobot platform with datarobot python client. 
client = dr.Client(token, "{}/api/v2".format(endpoint))
## grab mlops agents tarball
mlops_agents_tb = client.get("mlopsInstaller")
with open("/content/mlops-agent.tar.gz", "wb") as f:
    f.write(mlops_agents_tb.content)
## unpack tarball
os.system("tar -xf /content/mlops-agent.tar.gz -C . && mkdir -p /tmp/ta")

0

## Agents Framework

1. Scoring pipeline - used to score data
2. Python MLOps library - used to relay data to a buffer.  Buffer could be any of 
  * file system - used in this example
  * pubsub
  * kafka
  * sqs
  * rabbit
2. Tracking Agents (light weight java service) - monitors the buffer and relays data to DR MLOps


### Configure Tracking Agents

Very vanilla configuration


Using File system as a buffer

In [ ]:
import glob
agents_dir = glob.glob("/content/datarobot_mlops*").pop()
print(agents_dir)
with open('{}/conf/mlops.agent.conf.yaml'.format(agents_dir)) as file:
    documents = yaml.load(file)
## configure the loaction of the mlops instance with which we'll communicate
documents['mlopsUrl'] = endpoint
# Set your API token
documents['apiToken'] = token
## write the configuration back to disk
with open('{}/conf/mlops.agent.conf.yaml'.format(agents_dir), "w") as f:
    yaml.dump(documents, f)
## start the tracking agents service
subprocess.call("{}/bin/start-agent.sh".format(agents_dir))
## check status of agents service
check = subprocess.Popen(["{}/bin/status-agent.sh".format(agents_dir)], stdout=subprocess.PIPE)
print(check.stdout.readlines())
check.terminate()

/content/datarobot_mlops_package-8.0.5
[b'DataRobot MLOps-Agent is running as a service.\n']


In [ ]:
os.system('pip install {}/lib/datarobot_mlops-*.whl'.format(agents_dir))
os.system('pip install {}/lib/datarobot_mlops_connected_client-*.whl'.format(agents_dir))

0

## Upload Training Data

In [ ]:
upload_new_data = False

In [ ]:
if upload_new_data:
    print("Uploading training data. This may take some time...")
    training_data = dr.Dataset.create_from_in_memory_data(df.drop(["AssociationID"], axis=1))
    training_data.modify(name='Medical Shipments')
else:
    dataset_id = '61fcf913f2012b0d735aa891'
    training_data = dr.Dataset.get(dataset_id)

## Register Model, and Enable Monitoring with DataRobot MLOps

In [ ]:
register_new_model = False

In [ ]:
from datarobot.mlops.mlops import MLOps
from datarobot.mlops.connected.client import MLOpsClient

# take same model_info

DEPLOYMENT_NAME="Remote model in colab toon.weyens"
model_info = {
              "name": "Late shipment model",
              "modelDescription": {
                "description": "Binary classification on late shipment dataset",
                "location": "colab"
              },
              "target": {
                 "type": "Binary",
                 "name": "Late_delivery",
                 "classNames": ["1","0"],  # minority/positive class should be listed first
                 "predictionThreshold": 0.5
                }
            }
model_info["datasets"] = {"trainingDataCatalogId": training_data.id}

if register_new_model:
    # Create connected client
    mlops_connected_client = MLOpsClient(endpoint, token)
    # Create the model package
    print('Create model package')
    model_pkg_id = mlops_connected_client.create_model_package(model_info)
    model_pkg = mlops_connected_client.get_model_package(model_pkg_id)
    model_id = model_pkg["modelId"]
    # Deploy the model package
    print('Deploy model package')
    deployment_id = mlops_connected_client.deploy_model_package(model_pkg["id"], label = DEPLOYMENT_NAME)
else:
    deployment_id = "61fe425df01c67f2f5765e67"

In [ ]:
!mlops-cli --help

usage: mlops-cli [-h] [--version] [--verbose] [--terse]
                 [--mlops-url MLOPS_SERVICE_URL] [--api-token MLOPS_API_TOKEN]
                 [--verify-ssl VERIFY_SSL] [--deployment-id DEPLOYMENT_ID]
                 [--prediction-environment-id PREDICTION_ENVIRONMENT_ID]
                 [--model-id MODEL_ID] [--model-package-id MODEL_PACKAGE_ID]
                 [--target-col TARGET_COL]
                 [--prediction-cols PREDICTION_COLS [PREDICTION_COLS ...]]
                 [--class-names CLASS_NAMES [CLASS_NAMES ...]] [--dry-run]
                 [--status-file STATUS_FILE] [--assoc-id-col ASSOC_ID_COL]
                 [--json-config JSON_CONFIG] [--dataset-id DATASET_ID]
                 [--timeout TIMEOUT] [--search SEARCH] [--limit LIMIT]
                 [--input INPUT] [--count COUNT] [--rows ROWS]
                 [--prediction-time PREDICTION_TIME]
                 [--filesystem-directory FILESYSTEM_DIRECTORY] [--from-spool]
                 [--num-concurrent-r

## Update Drift Tracking

In [ ]:
deployment = dr.Deployment.get(deployment_id)
if register_new_model:
    deployment.update_drift_tracking_settings(target_drift_enabled=True, feature_drift_enabled=True)

## Update accuracy tracking

In [ ]:
if register_new_model:
    deployment.update_association_id_settings(column_names = ["AssociationID"], required_in_prediction_requests=False)

## Review deployment detail

In [ ]:
DEPLOYMENT_ID = deployment.id
MODEL_ID = deployment.model["id"]

deployment_details = client.get(f"deployments/{deployment.id}").json()

deployment_details_df = pd.DataFrame({"model name": deployment_details["label"],
 "model description": deployment_details["description"],
 "create date": deployment_details["createdAt"],
 "deployment id": deployment_details["id"], 
 "approval status": deployment_details["approvalStatus"], 
 "importance": deployment_details["importance"],
 "owners": deployment_details["owners"]}).T
 
deployment_details_df[["preview"]]

,preview
model name,Remote model in colab toon.weyens
model description,
create date,2022-02-05T09:24:46.051000Z
deployment id,61fe425df01c67f2f5765e67
approval status,APPROVED
importance,None
owners,"[{'id': '6156cfd4c5145f3407af633b', 'firstName..."


## Original Scoring Pipepline

In [ ]:
model = pd.read_pickle("/content/agents-plus-challengers/remote-model/artifact.pkl")
predictions = model.predict_proba(sampled_df.drop([target, "AssociationID"], axis=1)).values.tolist()
predictions[0:5]

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


[[0.9911746912143814, 0.008825308785618585],
 [0.89082891104194, 0.10917108895806006],
 [0.7540838825838062, 0.2459161174161938],
 [0.9873671809999274, 0.012632819000072576],
 [0.9932082180124516, 0.006791781987548334]]

## Modified Scoring Pipeline

In [ ]:
from datarobot.mlops.mlops import MLOps
# setting environment variables 
os.environ["MLOPS_DEPLOYMENT_ID"]= DEPLOYMENT_ID
os.environ["MLOPS_MODEL_ID"]= MODEL_ID
os.environ["MLOPS_SPOOLER_TYPE"]= "FILESYSTEM"
os.environ["MLOPS_FILESYSTEM_DIRECTORY"]= "/tmp/ta"
# init mlops client
mlops = MLOps().init()
# Load model
model = pd.read_pickle("/content/agents-plus-challengers/remote-model/artifact.pkl")
# start timeing
start_time = time.time()
############## original scoring pipeline ##############
predictions = model.predict_proba(sampled_df.drop([target, "AssociationID"],axis=1)).values.tolist()
#######################################################
# end timer
end_time = time.time()
# number of predictions
num_predictions = len(predictions)
# report stats
mlops.report_deployment_stats(num_predictions, end_time - start_time)
# report the predictions data: features, predictions
mlops.report_predictions_data(features_df=sampled_df.drop([target], axis=1), 
                                predictions=predictions, class_names = ["0", "1"])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


True

In [ ]:
# shutdown python client.  Java service is still running
mlops.shutdown()

In [ ]:
check = subprocess.Popen(["{}/bin/status-agent.sh".format(agents_dir)], stdout=subprocess.PIPE)
print(check.stdout.readlines())
check.terminate()

[b'DataRobot MLOps-Agent is running as a service.\n']


## Submitting Actuals

In [ ]:
actuals_for_submission = sampled_df[[target, "AssociationID"]].rename(columns = {target: "actual_value", "AssociationID": "association_id"})
deployment.submit_actuals(actuals_for_submission)

In [ ]:
print("https://app.datarobot.com/deployments/{}/overview".format(os.environ["MLOPS_DEPLOYMENT_ID"]))

https://app.datarobot.com/deployments/61fe425df01c67f2f5765e67/overview


In [ ]:
service_stats = deployment.get_service_stats()
service_stats.metrics
pd.DataFrame([service_stats.metrics]).T

,0
totalPredictions,1238.00000
totalRequests,2.00000
slowRequests,0.00000
executionTime,0.00988
responseTime,0.00000
userErrorRate,0.00000
serverErrorRate,0.00000
numConsumers,1.00000
cacheHitRatio,0.00000
medianLoad,0.00000


## Retrieve Accuracy metrics

In [ ]:
accuracy = deployment.get_accuracy_over_time()
metrics = ['AUC', 'FVE Binomial', 'Gini Norm', 'Kolmogorov-Smirnov', 'LogLoss', 'MCC', 'PPV', 'NPV', 'TPR', 'FPR', 'F1','Rate@Top10%', 'Rate@Top5%']
metrics_ls = []
for metric in metrics:
    m = deployment.get_accuracy_over_time(metric = metric)
    start = m.summary["period"]["start"]
    end = m.summary["period"]["end"]
    temp = [metric, m.baseline["value"], m.summary["value"], start, end, m.summary["sample_size"]]
    metrics_ls.append(temp)     
metrics_df = pd.DataFrame(metrics_ls, columns = ["metric", "baseline", "value", "start", "end", "n"])
metrics_df

,metric,baseline,value,start,end,n
0,AUC,None,0.914196,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00,1238
1,FVE Binomial,None,0.369527,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00,1238
2,Gini Norm,None,0.828393,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00,1238
3,Kolmogorov-Smirnov,None,0.710703,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00,1238
4,LogLoss,None,0.212900,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00,1238
5,MCC,None,0.391595,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00,1238
6,PPV,None,0.784757,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00,1238
7,NPV,None,0.915555,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00,1238
8,TPR,None,0.226315,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00,1238
9,FPR,None,0.007346,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00,1238


## Retrieve Drift Metrics

In [ ]:
feature_drift = deployment.get_feature_drift()
fd_ls = [[fd.name, fd.metric, fd.drift_score, fd.feature_impact, fd.period["start"], fd.period["end"]] for fd in feature_drift]
drift_df = pd.DataFrame(fd_ls, columns = ["feature", "drift_metric", "drift_score", "feature Importance", "start", "end"])
drift_df

,feature,drift_metric,drift_score,feature Importance,start,end
0,ID,psi,0.009426,1.000000,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00
1,Vendor,psi,0.044469,0.628645,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00
2,Vendor INCO Term,psi,0.002249,0.495845,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00
3,Fulfill Via,psi,0.000499,0.405302,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00
4,Pack Price,psi,0.005788,0.355843,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00
5,Line Item Quantity,psi,0.006470,0.288750,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00
6,Manufacturing Site,psi,0.062426,0.280061,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00
7,Unit Price,psi,0.003680,0.267768,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00
8,Item Description,psi,0.027205,0.234614,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00
9,Brand,psi,0.044480,0.226124,2022-01-30 09:00:00+00:00,2022-02-06 09:00:00+00:00
